# **1) Import all packages**

In [3]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import math
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split 
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.regularizers import l1,l2
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score,roc_curve
tf.random.set_seed(42)
seed(42)

# **2) Get Dataset**

In [ ]:
df= pd.read_csv('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Data/Data Raw/1.processed_columns_merged.csv')

In [ ]:
print(df.head())
print(df.shape)

   AMT_ANNUITY  AMT_CREDIT  ...  SK_ID_CURR_CNT_POS_CASH  SK_ID_CURR_CNT_INSTALL
0      24700.5    406597.5  ...                     19.0                    19.0
1      35698.5   1293502.5  ...                     28.0                    25.0
2       6750.0    135000.0  ...                      4.0                     3.0
3      29686.5    312682.5  ...                     21.0                    16.0
4      21865.5    513000.0  ...                     66.0                    66.0

[5 rows x 416 columns]
(356255, 416)


In [ ]:
meta_cols = ['SK_ID_CURR']
meta_df = df[meta_cols]
df.drop(columns=meta_cols, inplace=True)

In [ ]:
def process_dataframe(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """

    # Label encode categoricals
    print('Label encoding categorical features...')
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
    print('Label encoding complete.')

    return input_df, categorical_feats.tolist(), encoder_dict

In [ ]:
df, categorical_feats, encoder_dict = process_dataframe(input_df=df)

# Capture other categorical features not as object data types:
non_obj_categoricals = [
    'FONDKAPREMONT_MODE', 'HOUR_APPR_PROCESS_START', 'HOUSETYPE_MODE',
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
    'NAME_INCOME_TYPE', 'NAME_TYPE_SUITE', 'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE', 'STATUS', 'NAME_CONTRACT_STATUS_CAVG',
    'WALLSMATERIAL_MODE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_CONTRACT_TYPE_BAVG',
    'WEEKDAY_APPR_PROCESS_START_BAVG', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 
    'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE_BAVG', 
    'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 
    'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 
    'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'NAME_CONTRACT_STATUS_CCAVG' 
]
categorical_feats = categorical_feats + non_obj_categoricals

Label encoding categorical features...
Label encoding complete.


In [ ]:
# Extract target before scaling
labels = df.pop('TARGET')

In [ ]:
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0]
null_ratios = null_counts / len(df)

# Drop columns over x% null
null_thresh = .8
null_cols = null_ratios[null_ratios > null_thresh].index
df.drop(null_cols, axis=1, inplace=True)
print('Columns dropped for being over {}% null:'.format(100*null_thresh))
for col in null_cols:
    print(col)
    if col in categorical_feats:
        categorical_feats.pop(col)
    
# Fill the rest with the mean (TODO: do something better!)
# merged_df.fillna(merged_df.median(), inplace=True)
df.fillna(0, inplace=True)

Columns dropped for being over 80.0% null:
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
DAYS_FIRST_DRAWING
RATE_INTEREST_PRIMARY_PRVMAX
RATE_INTEREST_PRIVILEGED_PRVMAX
DAYS_FIRST_DRAWING_PRVMAX
RATE_INTEREST_PRIMARY_PRVMIN
RATE_INTEREST_PRIVILEGED_PRVMIN
DAYS_FIRST_DRAWING_PRVMIN
AMT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_POS_CURRENT
AMT_INST_MIN_REGULARITY
AMT_PAYMENT_CURRENT
CNT_DRAWINGS_ATM_CURRENT
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWINGS_POS_CURRENT
CNT_INSTALMENT_MATURE_CUM
AMT_DRAWINGS_ATM_CURRENT_CCMEAN
AMT_DRAWINGS_OTHER_CURRENT_CCMEAN
AMT_DRAWINGS_POS_CURRENT_CCMEAN
AMT_INST_MIN_REGULARITY_CCMEAN
AMT_PAYMENT_CURRENT_CCMEAN
CNT_DRAWINGS_ATM_CURRENT_CCMEAN
CNT_DRAWINGS_OTHER_CURRENT_CCMEAN
CNT_DRAWINGS_POS_CURRENT_CCMEAN
CNT_INSTALMENT_MATURE_CUM_CCMEAN
AMT_DRAWINGS_ATM_CURRENT_CCMAX
AMT_DRAWINGS_OTHER_CURRENT_CCMAX
AMT_DRAWINGS_POS_CURRENT_CCMAX
AMT_INST_MIN_REGULARITY_CCMAX
AMT_PAYMENT_CURRENT_CCMAX
CNT_DRAWINGS_ATM_CURRENT_CCMAX
CNT_DRAWINGS_OTHER_CURRE

In [ ]:
#Check LabelEncoding
print(df['CHANNEL_TYPE'])

0         8
1         8
2         7
3         8
4         7
         ..
356250    5
356251    8
356252    7
356253    8
356254    8
Name: CHANNEL_TYPE, Length: 356255, dtype: int64


In [ ]:
cat_feats_idx = np.array([df.columns.get_loc(x) for x in categorical_feats])
int_feats_idx = [df.columns.get_loc(x) for x in non_obj_categoricals]
cat_feat_lookup = pd.DataFrame({'feature': categorical_feats, 'column_index': cat_feats_idx})
cat_feat_lookup.head()

,feature,column_index
0,CODE_GENDER,18
1,EMERGENCYSTATE_MODE,32
2,FLAG_OWN_CAR,63
3,FLAG_OWN_REALTY,64
4,FONDKAPREMONT_MODE,73


In [ ]:
#Extract Continious Features list and index
cont_feats_idx = np.array(
    [df.columns.get_loc(x) 
     for x in df.columns[~df.columns.isin(categorical_feats)]]
)
cont_feat_lookup = pd.DataFrame(
    {'feature': df.columns[~df.columns.isin(categorical_feats)], 
     'column_index': cont_feats_idx}
)
cont_feat_lookup.head()

,feature,column_index
0,AMT_ANNUITY,0
1,AMT_CREDIT,1
2,AMT_GOODS_PRICE,2
3,AMT_INCOME_TOTAL,3
4,AMT_REQ_CREDIT_BUREAU_DAY,4


In [ ]:
#Save for features for later
#pd.DataFrame(cont_feat_lookup).to_csv("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Data/Data Clean/Final/continious_features_names.csv")
#pd.DataFrame(cat_feat_lookup).to_csv("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Data/Data Clean/Final/categorical_features_names.csv")

In [ ]:
#Scaling 
scaler = StandardScaler()
final_col_names = df.columns
df = df.values
df[:, cont_feats_idx] = scaler.fit_transform(df[:, cont_feats_idx])

scaler_2 = MinMaxScaler(feature_range=(0, 1))
df[:, int_feats_idx] = scaler_2.fit_transform(df[:, int_feats_idx])

In [ ]:
print(df.shape)
print(df.dtype)

(356255, 368)
float64


In [ ]:
#np.save('FINALDATA_Train_Test.npy', df)

In [ ]:
test = pd.read_csv('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Data/Data Raw//application_test.csv')
train = pd.read_csv('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Data/Data Raw/application_train.csv')

In [ ]:
print(train.shape)
print(test.shape)

(307511, 122)
(48744, 121)


In [ ]:
#The test labels are included in this dataset, however, they do not have labels due to the Kaggle competition. Therefore they have to be removed
print(test.shape)
length = len(train)
test_df = df[length:]
print(test_df.shape)

(48744, 121)
(48744, 368)


In [ ]:
train_df = df[:length]
print(train_df.shape)

(307511, 368)


In [ ]:
labels.head()
labels.shape

(356255,)

In [ ]:
train_labels = labels[:length]
test_labels = labels[length:]
print(train_labels.shape)

(307511,)


In [ ]:
#np.save('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Data/Data Clean/Final/Not_Upsampled/FINALDATA_TrainV2.npy',train_df)

In [ ]:
#Split dataset
X_train, X_test, y_train, y_test =  train_test_split(train_df, train_labels, train_size=0.85, test_size=0.15, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(261384, 368)
(261384,)


In [ ]:
#Save models for later use
np.save('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_train.npy',X_train)
np.save('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_trainlabels.npy',y_train)
np.save('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_test.npy',X_test)
np.save('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_testlabels.npy',y_test)

# **3) Baseline Model**

In [8]:
#Load data to increase speed for re-run
X_train = np.load('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_train.npy')
y_train = np.load('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_trainlabels.npy')
X_test = np.load('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_test.npy')
y_test = np.load('/content/drive/MyDrive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Preset Weights/Data/weights_testlabels.npy')

In [ ]:
log_X_train = X_train
log_y_train = y_train
log_X_test = X_test
log_y_test = y_test

In [ ]:
#Create Logistic Regression Model
logreg = LogisticRegression(max_iter= 3000, random_state=16)
logreg.fit(log_X_train,log_y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#Get Baseline score
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
print(logit_roc_auc)

0.5095394973685591


# **4) Vanilla Decision Tree**

In [30]:
#Create decision tree for specified depth of 5 and 10 
model = DecisionTreeClassifier(max_depth=5,random_state=16)
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=16, splitter='best')

In [31]:
#Get predictions of vanilla DT
res_pred = model.predict(X_test)

In [32]:
#Get various scores
acc = accuracy_score(y_test, res_pred)
rec = recall_score(y_test, res_pred,average='macro')
prec = precision_score(y_test, res_pred,average='macro')
auc = roc_auc_score(y_test,res_pred, average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.9194181282112429
Recall:  0.5
Precision: 0.45970906410562146
AUC:  0.5


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **5) Built Neural Network** 



In [6]:
#Import metrics for Neural Network
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def auc_m(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

In [ ]:
#Set class weights
eight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weight_dict = dict(enumerate(weight))

In [ ]:
print(class_weight_dict)

{0: 0.5439244868401338, 1: 6.191586128482092}


In [10]:
y_train = np.array(keras.utils.to_categorical(y_train))
y_test = np.array(keras.utils.to_categorical(y_test))
print(y_train.shape)
print(y_train[:10])

(261384, 2)
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [ ]:
#Function to built model
def build_model():
  inputs = layers.Input(shape=(368))
  x = layers.Dense(80,activation='relu',name='fc1')(inputs)

  x = layers.BatchNormalization()(x)

  x = layers.Dense(160,activation='relu',name='fc2')(x)

  x = layers.BatchNormalization()(x)

  x = layers.Dense(320,activation='relu',name='fc3')(x)

  x = layers.Dropout(rate=0.5, name='Dropout1')(x)

  x = layers.Dense(80,activation='relu',name='fc4')(x)

  x = layers.Dense(2,name='logits')(x)

  preds = layers.Activation('softmax',name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model

model = build_model()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 368)]             0         
_________________________________________________________________
fc1 (Dense)                  (None, 80)                29520     
_________________________________________________________________
batch_normalization_8 (Batch (None, 80)                320       
_________________________________________________________________
fc2 (Dense)                  (None, 160)               12960     
_________________________________________________________________
batch_normalization_9 (Batch (None, 160)               640       
_________________________________________________________________
fc3 (Dense)                  (None, 320)               51520     
_________________________________________________________________
Dropout1 (Dropout)           (None, 320)              

In [ ]:
#Set-up filepath and ModelCheckepoint with Earlystopping to save best performing moddel based on Validation AUC
filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Saved Models/Weights_MODELV1.hdf5"
model_checkpoint = ModelCheckpoint(filepath,  monitor='val_auc', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_auc', mode='max', verbose=0, patience=5)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC(name='auc')])

In [ ]:
#Set epochs and bach size, and train model 
num_epoch = 100
batch_size = 64
model1 = model.fit(x=X_train, y=y_train, class_weight = class_weight_dict, epochs=num_epoch, batch_size=batch_size, validation_split=0.2,callbacks=[model_checkpoint,es])

Epoch 1/50
3268/3268 [==============================] - 19s 6ms/step - loss: 0.6227 - acc: 0.6533 - f1_m: 0.6533 - precision_m: 0.6533 - recall_m: 0.6533 - auc: 0.7073 - val_loss: 0.4905 - val_acc: 0.7297 - val_f1_m: 0.7297 - val_precision_m: 0.7297 - val_recall_m: 0.7297 - val_auc: 0.8386
Epoch 2/50
3268/3268 [==============================] - 19s 6ms/step - loss: 0.5980 - acc: 0.6789 - f1_m: 0.6789 - precision_m: 0.6789 - recall_m: 0.6789 - auc: 0.7384 - val_loss: 0.4822 - val_acc: 0.7698 - val_f1_m: 0.7699 - val_precision_m: 0.7699 - val_recall_m: 0.7699 - val_auc: 0.8602
Epoch 3/50
3268/3268 [==============================] - 18s 6ms/step - loss: 0.5942 - acc: 0.6832 - f1_m: 0.6832 - precision_m: 0.6832 - recall_m: 0.6832 - auc: 0.7435 - val_loss: 0.5830 - val_acc: 0.6930 - val_f1_m: 0.6930 - val_precision_m: 0.6930 - val_recall_m: 0.6930 - val_auc: 0.7591
Epoch 4/50
3268/3268 [==============================] - 18s 6ms/step - loss: 0.5896 - acc: 0.6874 - f1_m: 0.6874 - precision_m:

In [ ]:
#Train performance
prediction_model = tf.keras.models.load_model(filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_train, y_train, verbose=0, batch_size=64)

print('Train Accuracy:',accuracy)
print('Train Recall:',recall)
print('Train Precision:',precision)
print('Train F1:',f1_score)
print('Train AUC:',auc)
prediction_model.summary()

Train Accuracy: 0.7721819281578064
Train Recall: 0.7722039222717285
Train Precision: 0.7722039222717285
Train F1: 0.7722038626670837
Train AUC: 0.8617718815803528
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 368)]             0         
_________________________________________________________________
fc1 (Dense)                  (None, 80)                29520     
_________________________________________________________________
batch_normalization_8 (Batch (None, 80)                320       
_________________________________________________________________
fc2 (Dense)                  (None, 160)               12960     
_________________________________________________________________
batch_normalization_9 (Batch (None, 160)               640       
_________________________________________________________________
fc3 (Dense)            

In [ ]:
#Test performance
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

print('Test Accuracy:',accuracy)
print('Test Recall:',recall)
print('Test Precision:',precision)
print('Test F1:',f1_score)
print('Test AUC:',auc)
prediction_model.summary()

Test Accuracy: 0.77232426404953
Test Recall: 0.7723062038421631
Test Precision: 0.7723062038421631
Test F1: 0.7723060846328735
Test AUC: 0.8606595993041992
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 368)]             0         
_________________________________________________________________
fc1 (Dense)                  (None, 80)                29520     
_________________________________________________________________
batch_normalization_8 (Batch (None, 80)                320       
_________________________________________________________________
fc2 (Dense)                  (None, 160)               12960     
_________________________________________________________________
batch_normalization_9 (Batch (None, 160)               640       
_________________________________________________________________
fc3 (Dense)                  (

In [11]:
#Best Performance
filepath_best = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Saved Models/Weights_MODELV1.hdf5"
best_model = tf.keras.models.load_model(filepath_best, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = best_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

#print('Best Accuracy:',accuracy)
#print('Best Recall:',recall)
#print('Best Precision:',precision)
#print('Best F1:',f1_score)
print('Best AUC:',auc)
best_model.summary()

Best AUC: 0.8606595993041992
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 368)]             0         
_________________________________________________________________
fc1 (Dense)                  (None, 80)                29520     
_________________________________________________________________
batch_normalization_8 (Batch (None, 80)                320       
_________________________________________________________________
fc2 (Dense)                  (None, 160)               12960     
_________________________________________________________________
batch_normalization_9 (Batch (None, 160)               640       
_________________________________________________________________
fc3 (Dense)                  (None, 320)               51520     
_________________________________________________________________
Dropout1 (Dropout)       

# **6) Generate Soft Targets**

In [12]:
#Load the best model
model = tf.keras.models.load_model(filepath_best, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m}) 

In [13]:
#Compile it again
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC()])
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 368)]             0         
_________________________________________________________________
fc1 (Dense)                  (None, 80)                29520     
_________________________________________________________________
batch_normalization_8 (Batch (None, 80)                320       
_________________________________________________________________
fc2 (Dense)                  (None, 160)               12960     
_________________________________________________________________
batch_normalization_9 (Batch (None, 160)               640       
_________________________________________________________________
fc3 (Dense)                  (None, 320)               51520     
_________________________________________________________________
Dropout1 (Dropout)           (None, 320)              

In [14]:
#Remove last softmax layer and predict again
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(X_train)

In [15]:
#Get the logits
soft_targets = model_logits

In [16]:
np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Home Credit Default Dataset/Data/Soft Targets/Logits.npy", model_logits)

# **Knowledge Distillation**

In [22]:
#Set up Decision Tree for Knowledge Distillation at given depth 
model = DecisionTreeRegressor(max_depth=5,random_state=16)

In [23]:
#Train Decision Tree
model.fit(X_train,soft_targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=16, splitter='best')

In [24]:
#Get predictions
res_pred = model.predict(X_test)

In [25]:
# Apply softmax function to output of Decision Tree
pred_argmax = np.zeros_like(res_pred)
pred_argmax[np.arange(len(res_pred)), res_pred.argmax(1)] = 1 

In [26]:
#Get scores of student decision tree
acc = accuracy_score(pred_argmax,y_test)
rec = recall_score(y_test, pred_argmax,average='macro')
prec = precision_score(y_test, pred_argmax,average='macro')
auc = roc_auc_score(y_test, pred_argmax,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.8528627484987101
Recall:  0.5972104024176409
Precision: 0.5714772992782347
AUC:  0.5972104024176408
